<a href="https://colab.research.google.com/github/joshheyer/CONAFOR/blob/main/2-Composites_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Developed for Google LLC by RedCastle Resources Inc
# https://www.redcastleresources.com/

# <img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'>  Lab 2: Create Landsat and Sentinel-2 Annual Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## 2.0: Overview and Introduction


This notebook introduces the compositing methods used by the USDA Forest Service Landscape Change Monitoring System (LCMS). It then produces composites for a small LCMS study area. The composites will be used to generate time series data that LCMS modeling algorithms will use to predict land use, land cover, and vegetation cover change in the subsequent modules.

**Learn more about [LCMS](https://apps.fs.usda.gov/lcms-viewer/home.html).**


All composites are created and visualized using the `geeViz` Python package. `geeViz` provides command-line tools for processing, analyzing, and visualizing Google Earth Engine (GEE) objects. For map output visualization, the primary advantage of `geeViz` is that it does not rely on IPython. This is essential, since all operational LCMS processing is typically performed outside Notebooks, but we need to be able to debug outputs and look at them on the map.

**Learn more about [geeViz](https://github.com/gee-community/geeViz/tree/master).**


### 2.0.1: Objective

#### This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

#### The steps performed include:

- Install necessary packages
- Test various parameters to create composites
    - Run with default parameters
    - Include Landsat 7 in image composites
    - Set cloud masking settings
    - Set cloud shadow masking settings
- Create and export composite time series to asset

#### Learning objectives:
- Users will create cloud and cloud shadow-free composites using Fmask and cloudScore for Landsat, s2Cloudless for Sentinel-2, and temporal dark outlier mask (TDOM).  



### 2.0.2: Before you begin

#### If you are working in Workbench: Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating.
This gives the Map Viewer a url in which to host the viewer we will be generating.
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below).

![workspace url](https://github.com/redcastle-resources/lcms-training/blob/main/img/workspace-url.png?raw=1)

#### Set a folder to use for all exports under `export_path_root`
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the pre-baked folder (where outputs have already been created).
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as is. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
* It will be something like `projects/projectID/assets/someFolder`. Use the projectID associated with your account.
* This folder does not have to already exist. If it does not exist, it will be created

In [1]:
workbench_url = 'https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/'
pre_baked_path_root  = 'projects/ee-jheyer2325/assets'
export_path_root = pre_baked_path_root
print('Done')

Done


#### Installation
Install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

Note which python packages and components of geeViz you are importing here.

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import pandas as pd
import inspect,os

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.2/583.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.5 MB/s eta 0:00:00
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13786 sha256=9f677a77bae1b43ffc0b085c51a560199b58c5354edc13595bd16f321649c7ca
  Stored in directory: /root/.cache/pip/wheels/e5/41/13/ebdef29165b9309ec4e235dbff19eca8b6759125b0924ad430
Successfully built simpledbf
Initializing GEE
No credentials found
Will attempt ee.Authenticate
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bPMfXmRXlJCwyolxG3vA6hTMrIN613t1xGikNgYDjAE&tc=e6ySRrVqcrqcC4j

#### Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab or Workbench, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

**Warning!!**

* **Read-only access is provided for all authenticated GEE users for pre-baked outputs**
* **If you are using the pre-baked output location (`export_path_root = pre_baked_path_root`), you will see errors for any operation that tries to write, delete, or change access permissions to any asset in the pre-baked output location.**
* **This is expected and will not stop you from successfully running this notebook - ignore these error messages should they appear.**


In [3]:
# Create folder
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
print(export_composite_collection)

# Create a collection
aml.create_asset(export_composite_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Make it public
# This will fail if you are using the pre_baked_path_root output location
aml.updateACL(export_composite_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

projects/ee-jheyer2325/assets/lcms-training_module-2_composites
Asset projects/ee-jheyer2325/assets/lcms-training_module-2_composites already exists
Updating permissions for:  projects/ee-jheyer2325/assets/lcms-training_module-2_composites
Done


## 2.1: Explore geeViz functions

Over the course of the examples in this module, you'll become more familiar with the parameters used to create image composites. The main function you will use is from the geeViz library and is called `getLandsatAndSentinel2HybridWrapper`.

For more documentation of the parameters in the function, refer to the [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py) example in the geeViz documentation.

You can view the [source code](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) for this function on the geeViz GitHub.

While the `getLandsatAndSentinel2HybridWrapper` will meet the needs of most compositing workflows, sometimes it is necessary to have more control over how composites are created. For these instances, you should use the underlying functions that help with pre-processing both Landsat or Sentinel-2 data:
* [getProcessedLandsatScenes](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L2678)
* [getProcessedSentinel2Scenes](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L2799)

You can inspect the code for these functions by following the links above to the function or by using the `print(inspect.getsource(LIBRARY_NAME.FUNCTION_NAME))` command. Run the code block below to inspect the function from the geeViz library that you will be using.

In [14]:
print(inspect.getsource(getImagesLib.getLandsatAndSentinel2HybridWrapper))

def getLandsatAndSentinel2HybridWrapper(\
  studyArea,
  startYear,
  endYear,
  startJulian,
  endJulian,
  timebuffer =  0,
  weights =  [1],
  compositingMethod = 'medoid',
  toaOrSR = 'TOA',
  includeSLCOffL7 = False,
  defringeL5 = False,
  applyQABand = False,
  applyCloudProbability = True,
  applyShadowShift = False,
  applyCloudScoreLandsat = False,
  applyCloudScoreSentinel2 = False,
  applyTDOMLandsat = True,
  applyTDOMSentinel2 = True,
  applyFmaskCloudMask = True,
  applyFmaskCloudShadowMask = True,
  applyFmaskSnowMask = False,
  cloudHeights = ee.List.sequence(500,10000,500),
  cloudScoreThresh = 20,
  performCloudScoreOffset = True,
  cloudScorePctl = 10,
  zScoreThresh = -1,
  shadowSumThresh = 0.35,
  contractPixels = 1.5,
  dilatePixels = 3.5,
  shadowSumBands = ['nir','swir1'],
  landsatResampleMethod = 'near',
  sentinel2ResampleMethod = 'aggregate',
  convertToDailyMosaics = True,
  runChastainHarmonization = True,
  correctIllumination = False,
  correctScale = 

## 2.2: Create Landsat + Sentinel-2 Composites

### 2.2.1: Create Composites using basic parameters

#### Set up map
The first thing you'll do is set up the map viewer where you'll inspect the outputs you generate. You won't view the map until later.

In [5]:
# Set proxy url
Map.proxy_url = workbench_url

#First clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

# reset port if necessary
Map.port = 1232

print('Done')

Done


#### Define parameters

**The first parameters you define will set the **area** and **time range** over which you want to compute annual composites.**

* ```studyArea``` - a featureCollection, feature or geometry to specify the bounds of our area of interest

* ```startJulian``` - the first julian date to include in annual composites, used to specify a particular season if desired
* ```endJulian``` - the last julian date to include in annual composites, used to specify a particular season if desired

    This supports wrapping for tropics and southern hemisphere. If you are using wrapping and the majority of the days occur in the second year, the system:time_start will default to June 1 of that year. Otherwise, all system:time_starts will default to June 1 of the given year.

* ```startYear``` - the first year for which to compute annual composites
* ```endYear``` - the last year for which to compute annual composites

You should provide at least 3 year for time series methods to work well. If you are providing pre-computed stats for cloud masking using cloudScore and TDOM, this does not matter.

Below, you'll use a 3-year window to explore how composites are made. After you determine which parameters are appropriate, you will create composites for the entire time window for which Landsat data are available.

In [15]:
# Define user parameters:

# Specify study area
paises = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
mexico = paises.filter(ee.Filter.stringContains('ADM0_NAME','Mexico'))
studyArea = paises.filter(ee.Filter.stringContains('ADM1_NAME','Tabasco'))

# studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary')

# Cuando es el date range mejor para Mexico? Para compuestos annuales?
# Update the startJulian and endJulian
startJulian = 1
endJulian = 365

# Specify start and end years for all analyses
startYear = 2009
endYear = 2011

# By default do not run the Temporal Dark Outlier Mask (TDOM) cloud shadow masking method
# We will come back to this
applyTDOMLandsat = False
applyTDOMSentinel2 = False
print('Done')

Done


#### Build composites
Using the parameters you specified above, you will create a series of image composites.

You can review all of the parameters, and their defaults, in the [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py).


In [16]:
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2
                                                          )

#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Only including SLC On Landsat 7
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


Note above that the default settings apply the [Fmask Cloud Mask](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1111) and [Fmask Shadow Mask](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1133) for Landsat, and apply [Cloud Probability](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L668) for Sentinel-2 imagery.

Follow the hyperlinks above to see how these functions are implemented in geeViz.

#### Add to Map
The `geeView` module of the `geeViz` package provides functions for creating an interactive map viewer outside of the Google Earth Engine Code Editor and Playground. Generally, `Map` functions in `geeView` follow the same syntax as javascript Earth Engine commands in the playground.

For more information and examples of geeView capabilities, visit the [geeView example](https://github.com/gee-community/geeViz/blob/master/examples/geeViewExample.py) in the geeViz github.

The interactive map viewer will appear below.

In [17]:
Map.clearMap()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Default Params {}-{}'.format(startJulian,endJulian),'True')
Map.addLayer(studyArea,{},'Study Area')
Map.centerObject(studyArea,10)
Map.turnOnInspector()
Map.view()

Adding layer: Default Params 1-365
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://twrlgjdgrwm-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspect the outputs

Look at the map that appeared above. You may need to click the button next to the layer titled "Default Params" in order to view the layer. Click the play button
![play button](https://github.com/redcastle-resources/lcms-training/blob/main/img/play-button.png?raw=1) to show a time lapse of all of the annual composites created.  

**It is clear the default parameters do not work very well in this area.**
Few, if any of the years displayed have complete images for the annual composite. Data are missing for large swaths of the image.

Though the results panel above indicates operations that were applied to create the composites, it doesn't tell you the full list of parameters used to create the composite. You can access the parameters that were used through the properties of the returned collection.

Run the code block below to print the parameters used and display them in a table. Uncomment the second line if you want to see what the raw properties attribute looks like.

In [18]:
#Print parameters used
#Parameters are stored in properties of returnd collection
props = processedScenes.toDictionary().getInfo()
display(pd.DataFrame(list(props.values()),index = list(props.keys()),columns = ['Value']))

print('Done')

,Value
applyCloudProbability,True
applyCloudScoreLandsat,False
applyCloudScorePlus,False
applyCloudScoreSentinel2,False
applyFmaskCloudMask,True
applyFmaskCloudShadowMask,True
applyFmaskSnowMask,False
applyQABand,False
applyShadowShift,False
applyTDOMLandsat,False


Done


### 2.2.2: Include Landsat 7
Since there are not that many images available in this area for these years, let's try adding Landsat 7. Our default is to exclude Landsat 7 from image composites because of the [Landsat 7 scan line correction failure](https://www.usgs.gov/landsat-missions/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con).

#### Add a new parameter

To include Landsat 7, you will add a new parameter to the wrapper function.

``` includeSLCoffL7 ``` - default: **False**

Run the code block below to add the new parameter.

In [19]:
#add LS 7 parameter
includeSLCOffL7 = True
print('Done')

Done


#### Call on wrapper function again, including new parameter

Now, run the same wrapper function again. Note that all of the parameters are the same, with the addition of your new `includeSLCOffL7` parameter.

In [20]:
#Call on master wrapper function to get Landsat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Including All Landsat 7
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


#### Add to map

Run the codeblock below to add the composites you created to the map again to inspect your results.

In [21]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 added {}-{}'.format(startJulian,endJulian),True)


# Stop map from centering on the study area
Map.mapCommandList = []

# Turn on the insepctor
Map.turnOnInspector()

Map.view()

Adding layer: L7 added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://twrlgjdgrwm-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspect the outputs
Turn on the layers in the map viewer if necessary.

Toggle back and forth between the Default Params and the new composites with Landsat 7 added. Zoom in to view the composites in more detail.

You'll notice this helps fill in the holes, but introduces many cloud-related artifacts.

## 2.3: Improve Cloud masking
Next you'll improve the cloud masking. [Fmask](https://www.sciencedirect.com/science/article/abs/pii/S0034425714005069) is used by default for Landsat, but this algorithm misses some clouds.

We'll try the adding in the [cloudScore](https://developers.google.com/earth-engine/guides/landsat#simple-cloud-score) method to mask clouds in Landsat.

We do this by setting the ```applyCloudScoreLandsat``` parameter

### 2.3.1: Add cloud masking parameter

Run the code block below to add the `applyCloudScoreLandsat` parameter.

CloudScore struggles with commission error over bright surfaces (masking out extremely bright cool areas as cloud). The `performCloudScoreOffset` method can reduce this error by finding if a pixel typically has a high cloudScore and adjust the cloudScore threshold for those pixels. This method works well in areas with many observations and potential for bright cool sufaces (cold deserts, snowy areas, etc), but fails in areas with limited data and/or areas that are actually always cloudy. Therefore we will want to set this to `False` over Puerto Rico and the US Virgin Islands

In [22]:
# Set cloudScore
applyCloudScoreLandsat = True

# Set performCloudScoreOffset
performCloudScoreOffset = False

print('Done')

Done


### Run main wrapper function

Keeping all other parameters the same, run the `getLandsatAndSentinel2HybridWrapper` function again. This time, add in the new parameters.

In [23]:
#Call on master wrapper function to get Landsat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat,
                                                            performCloudScoreOffset = performCloudScoreOffset)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Including All Landsat 7
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


### Add to Map

Run the code block below to add the new layers to the map.

In [ ]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()


Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 and CloudScore added {}-{}'.format(startJulian,endJulian),True)


Map.view()

Adding layer: L7 and CloudScore added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://bm9sk1jqt3-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


### Inspect
Toggle on the layers in the viewer above. Inspect the **L7 and cloudScore** output. Notice that these layers look much better and have fewer cloud masking artifacts than the previous layers.

However, you can still see some dark ares. These are likely due to cloud shadows that were not masked out-- the Fmask cloud shadow mask typically does not mask out all cloud shadows.

To mask cloud shadow, we use a method called the [Temporal Dark Outlier Mask (TDOM)](https://www.mdpi.com/2072-4292/10/8/1184) that works well to mask cloud shadows.



### 2.3.2: Use Temporal Dark Outlier Mask (TDOM) to mask cloud shadows
The Temporal Dark Outlier Mask (TDOM) method of masking cloud shadows calculates the mean and standard deviation of the NIR and SWIR bands over a pixel in a time series. The algorithm finds dark pixels ((SWIR1 + NIR) < 0.35) that are outliers (< -1 standard devitation) to determine which pixels are likely to be cloud shadows (dark pixels that aren't normally dark).

Find [details of the TDOM method](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L1141) in the geeViz library.

#### Set TDOM parameter

 - ``` applyTDOM ``` - default = **False**

Run the code block below to set the TDOM parameter.

In [24]:
#Set TDOM to true
applyTDOMLandsat = True
applyTDOMSentinel2 = True
print('Done')

Done


#### Run master wrapper function

Call on master wrapper function to get Landat scenes and composites
In order to identify dark outliers, we will extend the dates by 3 years in either direction to get a larger sample

In [25]:
#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 3 years in either direction to get a larger sample
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear - 3,
                                                           endYear + 3,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Get Processed Landsat: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Including All Landsat 7
Applying Cloud Score
Computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Running Chastain et al 2019 harmonization


### Add to map

Run the code block below to add the new layers to the map. This layer may take longer to run and load.

In [ ]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)

Map.view()

Adding layer: CloudScore and TDOM added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://bm9sk1jqt3-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspect
Toggle on the new layer. You'll notice this cleans up the cloud masking a lot.

#### TDOM is computationally intensive
However, while TDOM is a great cloud shadow masking method, it is a bit computationally intensive since it computes the mean and standard deviation over a large stack of data.

In order to avoid re-computing the stats, we store pre-computed stats. For the LCMS project, we have precomputed stats for CONUS, AK, HI, PuertoRico, and the USVI. We will be using these pre-computed stats


It is optional that you use pre-computed stats. If none are provided, TDOM stats will be computed on-the-fly.

You can create and store precomputed TDOM stats by only applying cloud masking over a period of time and computing the mean and standard deviation of NIR and SWIR1 bands.


#### Inspect raw, precomputed TDOM stats

Run the chuck of code below to load the precomputed TDOM stats and examine them on the map.

In [ ]:
# clear map
Map.clearMap()

# load precomputed tdom stats
landsat_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Landsat_TDOM_Stats_1984_2021')\
                    .select(['Landsat_nir_.*','Landsat_swir1_.*'])\
                    .divide(10000)
s2_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Sentinel2_TDOM_Stats_2015_2021')\
                    .select(['Sentinel2_nir_.*','Sentinel2_swir1_.*'])\
                    .divide(10000)
Map.addLayer(landsat_tdom_stats,{'min':0.15,'max':0.35,'bands':'Landsat_nir_mean','palette':'222,080'},'Landsat TDOM Stats')
Map.addLayer(s2_tdom_stats,{'min':0.15,'max':0.35,'bands':'Sentinel2_nir_mean','palette':'222,080'},'Sentinel 2 TDOM Stats')

Map.turnOnInspector()
Map.view()


Adding layer: Landsat TDOM Stats
Adding layer: Sentinel 2 TDOM Stats
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## 2.4: Prepare final composites and export
Now that you've gone through some of the parameters that can be changed for the composites, we will use both Landsat and Sentinel2 to get the best possible composites.

Here, you will manually set all parameters for the `getLandatAndSentinel2HybridWrapper`.

### Projections

Some common projections are accessed through `getImagesLib.common_projection`
LCMS uses the same snap grid (transform) and projection (crs) as the USGS NLCD. This is often called USGS Albers.
You can change this to any projection you'd like based on your study area using EPSG codes or WKT format.


### Set parameters and Export

Review the [documentation](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) to recall the default values for each of these parameters.

Note in the function below where the parameters are different from the defaults.

Run the code block below to prepare and export composites.

In [27]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(\

    # Study area and time
    studyArea = studyArea,
    startYear = 1984,
    endYear = 2022,
    startJulian = 152,
    endJulian = 151,

    # Moving window options
    timebuffer =  0,# If multi-year moving window is needed, use timebuffer > 0 (e.g. timebuffer = 1 will result in a 3 year moving window)
    weights =  [1],# To compliment the timebuffer, set weights for each year of the moving window. This enables the center year to have more weight (e.g. [1,3,1] would weight the center year if timebuffer = 1 3x over the buffer years)

    # Composite methods and input imagery
    compositingMethod = 'medoid', # Specify median or medoid. Medoid allows for storing only the spectral and ancillary bands without indices
    toaOrSR = 'TOA', # It is advised to only use TOA when combining Landsat and Sentinel-2 since the Sentinel-2 SR data in GEE are terrain-corrected and Landsat are not
    includeSLCOffL7 = True, # Whether to include Landsat 7 after May 2003. Scanline artifacts can be found in composites when True
    defringeL5 = True, # Whether to get rid of fringe edges found mostly on Landsat 4 and 5
    landsatCollectionVersion = 'C2', # To use Collection 1 or 2. Only use C1 if doing legacy work prior to 2022

    # Cloud and shadow masking
    # There will be a parameter for applying CloudScore+ to Sentinel-2 provided once it is available for all S2 data
    applyQABand = False, # Whether to apply the default Sentinel-2 QA mask - this method is terrible and should not be used
    applyCloudProbability = True,# Whether to apply S2Cloudless to Sentinel-2 data. This method works really well
    applyShadowShift = False, # This method is largely deprecated
    applyCloudScoreLandsat = True, # We use this since fMask struggles for cloud masking
    applyCloudScoreSentinel2 = False, # We typically do not use cloudScore for Sentinel-2 since S2Cloudless works so well
    applyTDOMLandsat = True, # We use TDOM for Landsat since the fMask cloud shadow masking method struggles a lot
    applyTDOMSentinel2 = True, # This is the only easily available Sentinel-2 cloud shadow masking method (as of Aug 2023)
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudHeights = ee.List.sequence(500,10000,500), # This compliments applyShadowShift and is largely deprecated,
    cloudScoreThresh = 10,# The threshold applied to the cloudScore for Landsat and/or Sentinel-2
    performCloudScoreOffset = False,# Set this to False in areas with limited data and/or limited bright cool surfaces
    cloudScorePctl = 10, # If performCloudScoreOffset is set to True, this is the percentile of the cloudScores used to see if an area typically has a high cloudScore
    zScoreThresh = -1, # The TDOM Z-score threshold for finding dark outliers
    shadowSumThresh = 0.35,# The sum of the shadowSumBands has to be below this to be considered dark
    contractPixels = 1.5,# Inward buffer pixel radius to get rid of salt and pepper
    dilatePixels = 3.5, # Outwoard buffer pixel radius to dilate the cloud shadow mask by
    shadowSumBands = ['nir','swir1'], # Bands to use for TDOM to find dark outliers
    cloudProbThresh = 40, # The S2Cloudless cloud probability threshold

    # Image processing
    landsatResampleMethod = 'bicubic',
    sentinel2ResampleMethod = 'bicubic',
    convertToDailyMosaics = True, # Important if the composite observation count is important since Sentinel-2 has overlap with its tiled outputs
    runChastainHarmonization = False, # Set to True in order to perform regression harmonization between Landsat and Sentinel 2
    correctIllumination = False, # Method for illuminating hill shadows. This is largely deprecated and not used
    correctScale = 250,

    # Export parameters
    exportComposites = True,# Whether to export resulting composites
    outputName = 'Landsat-Sentinel2-Hybrid',
    exportPathRoot = export_composite_collection,
    crs = getImagesLib.common_projections['NLCD_CONUS']['crs'],
    transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    scale = None,
    overwrite = False,# Whether to overwrite existing composites

    # Precomputed TDOM and cloudscore stats
    preComputedLandsatCloudScoreOffset = None,
    preComputedLandsatTDOMIRMean = None,
    preComputedLandsatTDOMIRStdDev = None,
    preComputedSentinel2CloudScoreOffset = None,
    preComputedSentinel2TDOMIRMean = None,
    preComputedSentinel2TDOMIRStdDev = None
                                                           )


print('Done')

Get Processed Landsat and Sentinel2 Scenes: 
Start date: May 31 1984 , End date: May 31 2023
Get Processed Landsat: 
Start date: May 31 1984 , End date: May 31 2023
Defringing L4 and L5
Including All Landsat 7
Setting resample method to  bicubic
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: May 31 1984 , End date: May 31 2023
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting resample method to  bicubic
Converting S2 data to daily mosaics
Applying Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1986_1986_152_151
Exporti

### Manage export tasks
These export tasks can take anywhere from a few hours to a few days to run, depending on the size of your study area. It can be helpful to manage and observe the tasks after you've set them running.

Below are some code snippets to use to manage and inspect the export tasks.

Link to documentation about these code snippets.

In [ ]:
# If you'd like to track the tasks, use this:
tml.trackTasks2()

# Danger!!! - Only use these methods if needed - they can stop and delete work
# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_composite_collection, type = 'imageCollection')

print('Done')

37 tasks ready 2023-11-08 22:57:42
2 tasks running 2023-11-08 22:57:42
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:24']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:24']


37 tasks ready 2023-11-08 22:57:47
2 tasks running 2023-11-08 22:57:47
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:30']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:30']


37 tasks ready 2023-11-08 22:57:53
2 tasks running 2023-11-08 22:57:53
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:36']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:36']


37 tasks ready 2023-11-08 22:57:58
2 tasks running 2023-11-08 22:57:58
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:41']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:41']




### View composites as they are completed
After your composites have been exported, you can view the composites in your folder.

In [ ]:
# View composites as they are completed
composites = ee.ImageCollection(export_composite_collection)

# If your composites have not yet exported, you can use the following line of code to inspect pre-computed composites instead
composites = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-2_composites')

print('Done')

Done


### Inspect the outputs

Explore the composites and some attributes that describe the quality of the composites.

Notice how the number of observations increases and the predominant sensor changes over time

In [ ]:
# clear the map
Map.clearMap()

# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.addTimeLapse(composites, getImagesLib.vizParamsFalse10k,'Composites')

# By looking at the Sensor that is used, you can see
# with the introduction of Landsat 8 in 2013 and then Sentinel 2 in 2016-2017,
# composites become much better quality
Map.addTimeLapse(composites.select('sensor'),{'min':4,'max':22,'palette':'088,808','classLegendDict':{'Landsat (Landsat 5 = 5, Landsat 7 = 7, etc...)':'088','Sentinel2 (Sentinel 2A = 21, Sentinel 2B = 22, etc....)':'808'},'queryDict':{4:'L4',5:'L5',7:'L7',8:'L8',9:'L9',21:'S2a',22:'S2b'}},'Sensor')
Map.addTimeLapse(composites.select('compositeObsCount'),{'min':3,'max':10,'palette':'D0D,0D0'},'Composite Observation Counts')
Map.turnOnInspector()
Map.view()
# Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm

Adding layer: Composites
Adding layer: Sensor
Adding layer: Composite Observation Counts
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## Lab 2 Challenge:

**For Qwiklabs users**, this will be assessed for completion in the "Check My Progress" portion at the end of Lab 2.

1. Create a cloud-free medoid composite time series from 2020-2023 for days 1-150 using the methods we used in this module.

    * Use the following as your study area:

    ```python
          ee.Geometry.Polygon(
                [[[-65.91075380517458, 18.265940533464065],
                  [-65.91075380517458, 18.236922027462644],
                  [-65.87350328637575, 18.236922027462644],
                  [-65.87350328637575, 18.265940533464065]]], None, False)
      ```
      <br>
   
   * Select the following bands:`["green","red","nir","swir1","swir2","NBR"]`

<br>

2. Extract the values from the three annual composites for a given point location.

    * Use a point with these coordinates: `([-65.88860948754763, 18.249964652264815])`
            
    * Use the following function: `g2p.extractPointValuesToDataFrame`
    <br><br>
    Example:
    ```python
          extracted_values = g2p.extractPointValuesToDataFrame(
          composite,
          ee.Geometry.Point([-65.88860948754763, 18.249964652264815]),
          scale=30,
          crs = "EPSG:5070",
          transform = None,
          reducer = ee.Reducer.first(),
          includeNonSystemProperties = False,
          includeSystemProperties=True
          )
    ```
<br>    

3. Save extracted values to a csv file.

   * Save csv to this path: `"/tmp/challenge/module_2_challenge_answer.csv"`
     * **Note: The path to the csv must exactly match the path above.**
    <br>
    
    * Create the `"/tmp/challenge"` folder if it does not already exist.
      
        Example:
    ```python
        out_csv = "/tmp/challenge/module_2_challenge_answer.csv"
        if not os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
    ```
<br>

4.  Check that the output csv exists.
    
    * Example:
    ```python
        print(os.path.exists(out_csv))
    ```
<br>

In [ ]:
# Put your challenge code here


## Done with Lab 2

Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm in module 3.